In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 6.3 MB/s eta 0:00:00


In [3]:
url = "https://failbondi.fail/?date="
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
months_max_days = { "01": 31, "02": 28, "03": 31, "04": 30, "05": 31, "06": 30, "07": 31, "08": 31, "09": 30, "10": 31, "11": 30, "12": 31 }
month_days = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11","12","13","14","15","16","17","18","19","20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"]

In [4]:
def get_html_from_url(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        # Creamos un objeto BeautifulSoup para poder navegar por el HTML.
        soup = BeautifulSoup(response.content, "html.parser")
        return soup
    return None


In [5]:
def scraping_vuelos(html, fecha):
    # 1. Extraer encabezados
    headers = [th.text.strip() for th in html.find('thead').find_all('th')]

    # 2. Extraer filas
    rows = []
    table_body = html.find('tbody')
    for tr in table_body.find_all('tr'):
        cells = [td.text.strip() for td in tr.find_all('td')]
        rows.append(cells)

    # 3. Crear DataFrame para manipular fácilmente
    df = pd.DataFrame(rows, columns=headers)
    df['fecha'] = fecha

    return df



In [6]:
import time
import random

def get_report_by_month(year_month, max_days):
    lista_dfs = []
    for i in range(max_days):
        date = year_month + "-" + month_days[i]
        url_link = url + date
        main_content = get_html_from_url(url_link, headers)
        # Ejecutar la función
        try:
            df_iteracion = scraping_vuelos(main_content, date)
            lista_dfs.append(df_iteracion)
        except Exception as e:
            print(f"Error en fecha {date}: {e}")


    df_month = pd.concat(lista_dfs, ignore_index=True)
    print(f"[{year_month}] - Filas obtenidas: {len(df_month)}")
    time.sleep(random.uniform(2, 5))

    return df_month


In [7]:
year = "2025"
lista_dfs = []

from datetime import datetime

for month, max_days in months_max_days.items():
    year_month = year + "-" + month
    inicio_peticion = time.time()
    lista_dfs.append(get_report_by_month(year_month, max_days))

    fin_peticion = time.time()
    duracion = fin_peticion - inicio_peticion
    hora_actual = datetime.now().strftime('%H:%M:%S')

    print(f"[{hora_actual}] Finalizado: {year_month} | Tiempo: {duracion:.2f}s")

[2025-01] - Filas obtenidas: 2303
[23:29:30] Finalizado: 2025-01 | Tiempo: 41.61s
[2025-02] - Filas obtenidas: 1968
[23:30:07] Finalizado: 2025-02 | Tiempo: 37.06s
[2025-03] - Filas obtenidas: 2064
[23:30:45] Finalizado: 2025-03 | Tiempo: 38.14s
[2025-04] - Filas obtenidas: 1386
[23:31:19] Finalizado: 2025-04 | Tiempo: 34.02s
[2025-05] - Filas obtenidas: 1424
[23:31:55] Finalizado: 2025-05 | Tiempo: 36.33s
[2025-06] - Filas obtenidas: 1429
[23:32:30] Finalizado: 2025-06 | Tiempo: 35.11s
[2025-07] - Filas obtenidas: 1703
[23:33:09] Finalizado: 2025-07 | Tiempo: 38.56s
[2025-08] - Filas obtenidas: 1682
[23:33:47] Finalizado: 2025-08 | Tiempo: 38.66s
[2025-09] - Filas obtenidas: 1543
[23:34:22] Finalizado: 2025-09 | Tiempo: 34.13s
[2025-10] - Filas obtenidas: 1660
[23:34:57] Finalizado: 2025-10 | Tiempo: 35.95s
[2025-11] - Filas obtenidas: 1521
[23:35:33] Finalizado: 2025-11 | Tiempo: 35.42s
[2025-12] - Filas obtenidas: 2059
[23:36:09] Finalizado: 2025-12 | Tiempo: 35.85s


In [11]:
import os
def exportar_datos(lista_maestra, nombre_archivo, formato='csv'):
    """
    Exporta una lista de listas a CSV o Excel.

    :param lista_maestra: La data a guardar.
    :param nombre_archivo: Nombre del archivo (sin extensión).
    :param formato: 'csv' o 'excel'.
    """
    # Convertimos la lista a un DataFrame de Pandas
    lista_plana = [registro for sublista in lista_maestra for registro in sublista]

        # 2. Creamos el DataFrame a partir de la lista ya normalizada
    df = pd.DataFrame(lista_plana)
    ruta = ''

    try:
        if formato.lower() == 'csv':
            ruta = f"{nombre_archivo}.csv"
            df.to_csv(ruta, index=False, encoding='utf-8-sig')
            print(f"✅ Archivo CSV guardado como: {ruta}")

        elif formato.lower() == 'excel':
            ruta = f"{nombre_archivo}.xlsx"
            df.to_excel(ruta, index=False, engine='openpyxl')
            print(f"✅ Archivo Excel guardado como: {ruta}")

        else:
            print("❌ Formato no soportado. Usa 'csv' o 'excel'.")

    except Exception as e:
        print(f"Error al exportar: {e}")
    else:
        return ruta

# --- Ejemplo de uso ---
ruta_archivo = exportar_datos(lista_dfs, "reporte_final_anual", formato='csv')
file_size = os.path.getsize(ruta_archivo)

print(f"\n¡Éxito! El archivo '{ruta_archivo}' ha sido creado. Tamaño: {file_size}")

✅ Archivo CSV guardado como: reporte_final_anual.csv

¡Éxito! El archivo 'reporte_final_anual.csv' ha sido creado. Tamaño: 749


_____
# Reporte